# Практична робота №4
## Варіант 2
**Опис Компанії-Замовника:** Scott Polar Research Institute (SPRI) -  Британський інститут, що є частиною Кембриджського університету, який займається всесторонніми дослідженнями полярних регіонів.\
***---------------------------------------------***\
**Запит:** Організація потребує розробки алгоритму, здатного класифікувати різні види пінгвінів на основі зібраних даних, щоб поліпшити розуміння різноманіття та розподілу цих видів у регіоні.\
***---------------------------------------------***\
**Пропозиція Реалізації:** Створення моделі машинного навчання для класифікації видів пінгвінів. Модель повинна враховувати різні фізичні характеристики, такі як розміри тіла, маса та гендерні особливості, для точного визначення виду кожного пінгвіна. Ця класифікація допоможе визначити основні видові групи пінгвінів у регіоні та їх розподіл, сприяючи кращому збереженню цих видів та розумінню їхнього екологічного статусу.**

## Завдання
1) Виберіть декілька метрик для ваших моделей для оптимізації гіперпараметрів.
2) Виберіть метод оптимізації гіперпараметрів, гіперпараметри та їх можливі значення. Опишіть основи такого рішення.
3) Здійсніть оптимізацію гіперпараметрів моделі на основі обраних метрик та гіперпараметрів.
4) Оберіть оптимальний результат.
5) Поясніть результати пошуку гіперпараметрів моделі відповідно до ваших метрик.
6) Застосуйте підходи роботи з незбалансованими класами якщо є така необхідність. Обгрунтуйте свій вибір.



## Підготовка роботи ##

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score, f1_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
ds = pd.read_csv("variant_2_updated.csv")
X = ds.drop(columns=['Species', 'Unnamed: 0', 'Sample Number', 'Individual ID', 'studyName', 'Comments'])
y = ds['Species']

***Розділяємо дані на навчальну та тестову вибірки***

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

***З попередньої практичної визначили, що використовуватимемо модель логістичної регресії***

In [4]:
model = LogisticRegression(solver='liblinear')

***Для методу пошуку найкращих гіперпараметрів, обираю Grid Search та встановлюю йому початкові параметри***

In [5]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2']
}

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1': make_scorer(f1_score, average='weighted'),
    'roc_auc': make_scorer(roc_auc_score, needs_proba=True, multi_class='ovr')
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, refit='f1', cv=5)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Найкращі параметри:", best_params)

best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test), multi_class='ovr')

print("Точність:", accuracy)
print("F1-міра:", f1)
print("AUC-ROC:", roc_auc)

C:\Users\aziya\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(


Найкращі параметри: {'C': 1, 'penalty': 'l1'}
Точність: 0.989010989010989
F1-міра: 0.989008611980708
AUC-ROC: 0.9981080151358789


***Результати демонструють, що модель не має значного перенавчання, але щоби впевнитись, проведемо крос-валідацію.***

In [6]:
from sklearn.model_selection import cross_val_score
import numpy as np
model = LogisticRegression(C=1, penalty='l1', solver='liblinear')

f1_scorer = make_scorer(f1_score, average='weighted')
f1_scores = cross_val_score(model, X, y, cv=5, scoring=f1_scorer)

print("F1-міра на кожній ітерації крос-валідації:", f1_scores)
print("Середнє значення F1-міри:", np.mean(f1_scores))
print("Стандартне відхилення F1-міри:", np.std(f1_scores))

F1-міра на кожній ітерації крос-валідації: [1.         0.97799753 1.         0.9888858  1.        ]
Середнє значення F1-міри: 0.9933766664891148
Стандартне відхилення F1-міри: 0.008812392651665562


***Результати крос-валідації та тестування виглядають дуже добре і показують, що модель має дуже високу продуктивність. Тому можна вважатти, що параметри C=1 і penalty='l1' виявилися оптимальними, і їх можна залишити без змін***

***Перевіримо чи класи збалансовані***

In [7]:
print(y.value_counts())

Species
1    151
2    151
0    150
Name: count, dtype: int64


***Оскільки значення приблизно однакові, можна вважати, що класи збалансовані***

### Висновок

***Виконавши практичну роботу, я навчився оптимізовувати модель машинного навчання шляхом пошуку гіперпараметрів для покращення точності моделі.***